In [7]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import ReduceLROnPlateau, Callback
import os
from PIL import Image, UnidentifiedImageError, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


train_dataset_path = 'train_tomato'
validation_dataset_path = 'valid_tomato'

# Load the datasets
train_dataset = image_dataset_from_directory(
    train_dataset_path,
    image_size=(224, 224),
    batch_size=64,
    label_mode='int'
)
validation_dataset = image_dataset_from_directory(
    validation_dataset_path,
    image_size=(224, 224),
    batch_size=64,
    label_mode='int'
)



Found 25850 files belonging to 11 classes.
Found 6683 files belonging to 11 classes.


In [8]:
class_names = train_dataset.class_names

# Learning rate scheduler
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

class ImageLogger(tf.keras.callbacks.Callback):
    def __init__(self, log_dir):
        super().__init__()
        self.log_dir = log_dir
        self.writer = tf.summary.create_file_writer(log_dir)

    def on_batch_end(self, batch, logs=None):
        if logs and logs.get('loss') is None:  # Check if loss is None, which might indicate a problem
            with self.writer.as_default():
                tf.summary.text('Problematic Batch', f'Batch {batch} encountered an issue', step=batch)
                # You can add more logging here, such as saving the actual images if accessible

    def on_train_end(self, logs=None):
        self.writer.close()
image_logger = ImageLogger(log_dir='./image_logs')

# Build the model (same as before)
model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=8,
    callbacks=[lr_scheduler, image_logger], 
    verbose=1
)

# Evaluate the model
model.evaluate(validation_dataset, verbose=2)

# Save the model
model.save('tomato_final')


Epoch 1/8


2024-09-14 19:22:02.152714: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 6190353686219760102
2024-09-14 19:22:02.152732: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 6952222542046581724
2024-09-14 19:22:02.152736: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]]
	 [[StatefulPartitionedCall/sequential_1_1/dropout_3_1/stateless_dropout/stateless_random_uniform/StatelessRandomGetKeyCounter/_78]]
2024-09-14 19:22:02.152747: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2914046606119890808
2024-09-14 19:22:02.152755: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 17855890933744541869
202

InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) INVALID_ARGUMENT:  Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]]
	 [[StatefulPartitionedCall/sequential_1_1/dropout_3_1/stateless_dropout/stateless_random_uniform/StatelessRandomGetKeyCounter/_78]]
  (1) INVALID_ARGUMENT:  Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_one_step_on_iterator_9634]

In [27]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau
import os
from PIL import Image

def resize_large_images(directory, target_size=(224, 224)):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.png'):
                file_path = os.path.join(root, file)
                with Image.open(file_path) as img:
                    if img.size[0] > target_size[0] or img.size[1] > target_size[1]:
                        print(f"Resizing large image: {file}")
                        img = img.resize(target_size)
                        img.save(file_path)  # Overwrite the original file

# Function to filter allowed image formats (JPEG, PNG, GIF, BMP)
def filter_image_formats(directory):
    allowed_extensions = ['jpeg', 'png', 'gif', 'bmp']
    for root, dirs, files in os.walk(directory):
        for file in files:
            if not any(file.lower().endswith(ext) for ext in allowed_extensions):
                print(f"Removing unsupported file format: {file}")
                os.remove(os.path.join(root, file))

train_dataset_path = 'tomatodataset/train'
validation_dataset_path = 'tomatodataset/valid'

# # Filter files in both training and validation directories
# filter_image_formats(train_dataset_path)
# filter_image_formats(validation_dataset_path)

# # Apply resizing to both training and validation directories
# resize_large_images(train_dataset_path)
# resize_large_images(validation_dataset_path)

print("Done filtering and rezising")

train_dataset = image_dataset_from_directory(
    train_dataset_path,
    image_size=(224, 224),
    batch_size=128
)

validation_dataset = image_dataset_from_directory(
    validation_dataset_path,
    image_size=(224, 224),
    batch_size=128
)

class_names = train_dataset.class_names

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',  # Monitor the validation loss to adjust the learning rate
    factor=0.5,          # Reduce the learning rate by a factor of 0.5
    patience=3,          # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6,         # Lower bound on the learning rate
    verbose=1            # Print updates about learning rate reduction
)

model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=8,
    callbacks=[lr_scheduler] 
)

model.evaluate(validation_dataset, verbose=2)

# model.save('rice_model.h5')  # Saves to HDF5 format
model.export('tomato_final')

Done filtering and rezising


NotFoundError: Could not find directory tomatodataset/train

In [28]:
import openvino as ov
from openvino.tools import mo
from openvino.runtime import serialize

# Path to your SavedModel directory
saved_model_path = "tomato_final"

ov_model = mo.convert_model(saved_model_path, 
                            model_name="tomato_model",
                            framework="tf", input_shape=[1, 224, 224, 3])  

# Specify the output directory and filenames
output_dir = "openvino_model"
xml_filename = "tomato_model.xml"
bin_filename = "tomato_model.bin"

# Serialize the model to IR format
serialize(ov_model, xml_filename, bin_filename)

print(f"Model converted and saved to {output_dir}/{xml_filename} and {output_dir}/{bin_filename}")

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
Model converted and saved to openvino_model/tomato_model.xml and openvino_model/tomato_model.bin


In [30]:
import openvino as ov
from openvino.tools import mo
from openvino.runtime import serialize

# Path to your SavedModel directory
saved_model_path = "cucumber_final"

ov_model = mo.convert_model(saved_model_path, 
                            model_name="cucumber_model",
                            framework="tf", input_shape=[1, 224, 224, 3])  

# Specify the output directory and filenames
output_dir = "openvino_model"
xml_filename = "cucumber_model.xml"
bin_filename = "cucumber_model.bin"

# Serialize the model to IR format
serialize(ov_model, xml_filename, bin_filename)


print(f"Model converted and saved to {output_dir}/{xml_filename} and {output_dir}/{bin_filename}")

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html


OpConversionFailure: Check 'is_conversion_successful' failed at src/frontends/tensorflow/src/frontend.cpp:479:
FrontEnd API failed with OpConversionFailure:
[TensorFlow Frontend] Internal error, no translator found for operation(s): ImageProjectiveTransformV3, StatelessRandomGetKeyCounter, StatelessRandomUniformV2
To facilitate the conversion of unsupported operations, refer to Frontend Extension documentation: https://docs.openvino.ai/latest/openvino_docs_Extensibility_UG_Frontend_Extensions.html 

